In [87]:
import pandas as pd
import numpy as np
import os

from glob import glob
import yaml
import re
import copy
import json
import requests
from tqdm import tqdm

- OWL ontologies
- Websites:
  - OLS
  - Ontobee


# Functions


In [88]:
# Clean list columns into single string
def join_strings(string):
    try:
        return ','.join(string)
    except:
        return ''


validation_coder = {
    'number': 'regex search ([0-9]+\.[0-9]*.?)|([0-9]+)',
    'integer': 'regex search ([0-9]+)',
    'string': ''
}

In [89]:
# parse url into api call
def url_to_api_call(url):
    url = url.replace("ols4", "ols4/api")

    test = urllib.parse.urlparse(url)

    url_parts = list(urllib.parse.urlparse(url))

    iri = urllib.parse.parse_qs(test.query)['iri'][0]

    new_iri = urllib.parse.quote_plus(urllib.parse.quote_plus(iri))

    url_parts[2] = url_parts[2].replace('classes', 'terms')

    url_parts[2] = '/'.join([url_parts[2], new_iri])

    url_parts[4] = ''

    new_url = urllib.parse.urlunparse(url_parts)

    return new_url

In [90]:
from requests.exceptions import HTTPError


def get_response(u, params=None):
    """_summary_

    Args:
        u (_type_): _description_
        params (_type_, optional): _description_. Defaults to None.

    Returns:
        _type_: _description_
    """
    try:
        response = requests.get(u, params=params)

        # If the response was successful, no Exception will be raised
        response.raise_for_status()

    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')  # Python 3.6
    except Exception as err:
        print(f'Other error occurred: {err}')  # Python 3.6
    else:
        print(response)
        return response

In [91]:
dm = pd.read_csv(
    "C:/Users/nlee/Documents/Projects/ELITE-DCC/ELITE-data-models/models/EL_data_model_v3.csv")

In [92]:
dm_test = dm[dm['Valid Values'].fillna("").str.contains(
    'not listed for purposes')].reset_index(drop=True)

dm_test

,Unnamed: 0,Attribute,Description,Valid Values,DependsOn,Properties,Required,Parent,DependsOn Component,Source,Validation Rules
0,143,organ,Indicate the organ the specimen is from,NCI values (not listed for purposes of this RF...,NaN,dataProperty,True,Biospecimen human,NaN,http://purl.obolibrary.org/obo/NCIT_C13018,NaN
1,144,tissue,Indicate the tissue the specimen is from,NCI values (not listed for purposes of this RF...,NaN,dataProperty,True,Biospecimen human,NaN,http://purl.obolibrary.org/obo/NCIT_C12801,NaN
2,147,specimenSource,Indicate the biological area of the specimen,Human Reference Atlas values (not listed for p...,NaN,dataProperty,True,Biospecimen human,NaN,https://www.ebi.ac.uk/ols/ontologies/ccf/terms...,NaN
3,149,cellType,Indicate the cell type.,CCF-BSO values (not listed for purposes of thi...,NaN,dataProperty,True,Biospecimen human,NaN,https://bioportal.bioontology.org/ontologies/C...,NaN
4,162,countryCode,Indicate the geographic region for the individ...,World Bank country codes (not listed for purpo...,NaN,dataProperty,True,Individual Human,NaN,https://wits.worldbank.org/countryprofile/meta...,regex search ([0-9]+)
5,171,diagnosis,Indicate the disease or condition.,HPO and MONDO and MAXO codes or labels (not li...,NaN,dataProperty,False,Individual Human,NaN,"https://www.ebi.ac.uk/ols/ontologies/mondo,htt...",NaN
6,172,taxon,Taxonomic identifier. The National Center for ...,NCBI taxonomy values (not listed for purposes ...,NaN,dataProperty,True,Individual nonHuman,NaN,"https://www.ncbi.nlm.nih.gov/taxonomy,http://e...",NaN
7,173,speciesName,The scientific name of the species (typically ...,NCBI taxonomy values (not listed for purposes ...,NaN,dataProperty,True,Individual nonHuman,NaN,https://www.ebi.ac.uk/ols/ontologies/edam/term...,NaN
8,174,commonName,The biological species common name the individ...,Genbank common names (not listed for purposes ...,NaN,dataProperty,True,Individual nonHuman,NaN,https://www.ebi.ac.uk/ols/ontologies/edam/term...,NaN
9,219,TRUE,Indicate the disease or condition.,HPO and MONDO and MAXO codes or labels (not li...,diagnosis,validValue,False,Individual Human,NaN,"https://www.ebi.ac.uk/ols/ontologies/mondo,htt...",NaN


In [93]:
dm_test['Source'] = dm_test['Source'].str.replace("ols", "ols4", regex=True)
dm_test['Source'] = dm_test['Source'].str.replace(
    "terms", "terms?", regex=True)

In [94]:
with pd.option_context("display.max_colwidth", None):
    display(dm_test)

,Unnamed: 0,Attribute,Description,Valid Values,DependsOn,Properties,Required,Parent,DependsOn Component,Source,Validation Rules
0,143,organ,Indicate the organ the specimen is from,"NCI values (not listed for purposes of this RFC),Other,Unknown,Not collected,Not applicable",NaN,dataProperty,True,Biospecimen human,NaN,http://purl.obolibrary.org/obo/NCIT_C13018,NaN
1,144,tissue,Indicate the tissue the specimen is from,"NCI values (not listed for purposes of this RFC),Other,Unknown,Not collected,Not applicable",NaN,dataProperty,True,Biospecimen human,NaN,http://purl.obolibrary.org/obo/NCIT_C12801,NaN
2,147,specimenSource,Indicate the biological area of the specimen,"Human Reference Atlas values (not listed for purposes of this RFC),Other,Unknown,Not collected,Not applicable",NaN,dataProperty,True,Biospecimen human,NaN,https://www.ebi.ac.uk/ols4/ontologies/ccf/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FUBERON_0004537&lang=en&viewMode=PreferredRoots&siblings=false,NaN
3,149,cellType,Indicate the cell type.,"CCF-BSO values (not listed for purposes of this RFC),Other,Unknown,Not collected,Not applicable",NaN,dataProperty,True,Biospecimen human,NaN,"https://bioportal.bioontology.org/ontologies/CCF/p=classes&conceptid=http%3A%2F%2Fpurl.org%2Fccf%2Fcell_type,https://www.ebi.ac.uk/ols4/ontologies/ccf/terms?iri=http%3A%2F%2Fpurl.org%2Fccf%2Fcell_type",NaN
4,162,countryCode,"Indicate the geographic region for the individual (shown as ex. ""840"")","World Bank country codes (not listed for purposes of this RFC),Other,Unknown,Not collected,Not applicable",NaN,dataProperty,True,Individual Human,NaN,https://wits.worldbank.org/countryprofile/metadata/en/country/all,regex search ([0-9]+)
5,171,diagnosis,Indicate the disease or condition.,"HPO and MONDO and MAXO codes or labels (not listed for purposes of this RFC),Other,Unknown,Not collected,Not applicable",NaN,dataProperty,False,Individual Human,NaN,"https://www.ebi.ac.uk/ols4/ontologies/mondo,https://www.ebi.ac.uk/ols4/ontologies/maxo,https://www.ebi.ac.uk/ols4/ontologies/hp",NaN
6,172,taxon,"Taxonomic identifier. The National Center for Biotechnology Information taxon ID (formatted as, ex. ""txid9989"")","NCBI taxonomy values (not listed for purposes of this RFC),Unknown,Not collected,Not applicable",NaN,dataProperty,True,Individual nonHuman,NaN,"https://www.ncbi.nlm.nih.gov/taxonomy,http://edamontology.org/data_1179,http://purl.obolibrary.org/obo/TAXRANK_0000010",NaN
7,173,speciesName,"The scientific name of the species (typically a taxonomic group, ex. ""Eremophila alpestris"") the individual belongs to.","NCBI taxonomy values (not listed for purposes of this RFC),Unknown,Not collected,Not applicable",NaN,dataProperty,True,Individual nonHuman,NaN,https://www.ebi.ac.uk/ols4/ontologies/edam/terms?iri=http%3A%2F%2Fedamontology.org%2Fdata_1045,NaN
8,174,commonName,"The biological species common name the individual belongs to (ex. ""Horned Lark"").note: As a default, the valid scientific name for the species should be indicated.","Genbank common names (not listed for purposes of this RFC)Unknown,Not collected,Not applicable",NaN,dataProperty,True,Individual nonHuman,NaN,https://www.ebi.ac.uk/ols4/ontologies/edam/terms?iri=http%3A%2F%2Fedamontology.org%2Fdata_1874,NaN
9,219,TRUE,Indicate the disease or condition.,"HPO and MONDO and MAXO codes or labels (not listed for purposes of this RFC),Other,Unknown,Not collected,Not applicable",diagnosis,validValue,False,Individual Human,NaN,"https://www.ebi.ac.uk/ols4/ontologies/mondo,https://www.ebi.ac.uk/ols4/ontologies/maxo,https://www.ebi.ac.uk/ols4/ontologies/hp",NaN


# OLS

https://www.ebi.ac.uk/ols4/help  
replace ols with ols4


In [95]:
# to use for searching ols4 for terms
base_url = 'http://www.ebi.ac.uk/ols4/api/terms'

## Functions


purl case


In [96]:
def simplify_response(response):
    """ Get the terms from the response. Helps to simplify parsing the json """

    j = response.json()['_embedded']['terms']

    if len(j) == 1:
        return j[0]
    else:
        print(f"Response has {len(j)} terms")
        return j

In [97]:
def iri_request(iri):

    response = get_response(base_url, params={'iri': iri})

    return response

In [98]:
import urllib


def get_iri(url):

    test = urllib.parse.urlparse(url)

    params = urllib.parse.parse_qs(test.query)

    iri = params['iri'][0]

    return iri

In [99]:
def find_defining_ontology(terms):
    # look for defining ontology
    if len(terms) > 1:
        for t in terms:
            if t['is_defining_ontology'] == True:
                return t
            else:
                pass
    elif len(terms) == 1:
        return terms[0]
    else:
        return None

# term = find_defining_ontology(terms)
# term

In [100]:
def get_all_terms(url, json_response):
    # get all terms from pages
    pages = json_response['page']['totalPages']
    size = json_response['page']['size']

    # get the original link
    # url = json_response['_links']['self']['href']

    terms_list = []

    for i in tqdm(range(pages)):
        response = requests.get(url, params={'page': i, 'size': size})

        temp_d = response.json()

        for term in temp_d['_embedded']['terms']:
            terms_list.append(term)

    print("Length of terms: ", len(terms_list), sep="\t")

    return terms_list


# terms_list = get_all_terms(url, json_response)

In [101]:
def get_labels_from_terms_list(terms_list):
    vv = []

    for t in terms_list:
        # if t['has_children'] == False:
        vv.append(t)

    labels = []

    for i in vv:
        labels.append(i['label'])

    labels = sorted(np.unique(labels))

    return labels

In [102]:
def purl_main(iri):
    response = iri_request(iri)
    json_response = response.json()
    terms = json_response['_embedded']['terms']
    term = find_defining_ontology(terms)

    # get descendants
    url = term['_links']['hierarchicalDescendants']['href']
    d = get_response(url)
    json_response = d.json()

    terms_list = get_all_terms(url, json_response)

    valid_values = get_labels_from_terms_list(terms_list)

    return valid_values

# Start extraction


In [103]:
with pd.option_context('display.max_colwidth', 0):
    display(
        dm_test.iloc[4, :]
    )

Unnamed: 0             162                                                                                                      
Attribute              countryCode                                                                                              
Description            Indicate the geographic region for the individual (shown as ex. "840")                                   
Valid Values           World Bank country codes (not listed for purposes of this RFC),Other,Unknown,Not collected,Not applicable
DependsOn              NaN                                                                                                      
Properties             dataProperty                                                                                             
Required               True                                                                                                     
Parent                 Individual Human                                                          

In [104]:
dm_test['Source'] = dm_test['Source'].str.replace(
    "https://www.ebi.ac.uk/ols4/ontologies/mondo|https://www.ebi.ac.uk/ols4/ontologies/maxo|https://www.ebi.ac.uk/ols4/ontologies/hp", "LLFS data dictionary", regex=True)

In [105]:
dm_test['Source'] = dm_test['Source'].str.split(",")

In [106]:
dm_test['Valid Values'] = dm_test['Valid Values'].str.split(",")

In [107]:
# removing the "not listed for purposes of this RFC" values
for i, v in dm_test.iterrows():
    new_list = []
    for j in v['Valid Values']:
        if "not listed for purposes of this RFC" not in j:
            new_list.append(j)
        else:
            pass
        dm_test.loc[i, 'Valid Values'] = ','.join(new_list)

dm_test

,Unnamed: 0,Attribute,Description,Valid Values,DependsOn,Properties,Required,Parent,DependsOn Component,Source,Validation Rules
0,143,organ,Indicate the organ the specimen is from,"Other,Unknown,Not collected,Not applicable",NaN,dataProperty,True,Biospecimen human,NaN,[http://purl.obolibrary.org/obo/NCIT_C13018],NaN
1,144,tissue,Indicate the tissue the specimen is from,"Other,Unknown,Not collected,Not applicable",NaN,dataProperty,True,Biospecimen human,NaN,[http://purl.obolibrary.org/obo/NCIT_C12801],NaN
2,147,specimenSource,Indicate the biological area of the specimen,"Other,Unknown,Not collected,Not applicable",NaN,dataProperty,True,Biospecimen human,NaN,[https://www.ebi.ac.uk/ols4/ontologies/ccf/ter...,NaN
3,149,cellType,Indicate the cell type.,"Other,Unknown,Not collected,Not applicable",NaN,dataProperty,True,Biospecimen human,NaN,[https://bioportal.bioontology.org/ontologies/...,NaN
4,162,countryCode,Indicate the geographic region for the individ...,"Other,Unknown,Not collected,Not applicable",NaN,dataProperty,True,Individual Human,NaN,[https://wits.worldbank.org/countryprofile/met...,regex search ([0-9]+)
5,171,diagnosis,Indicate the disease or condition.,"Other,Unknown,Not collected,Not applicable",NaN,dataProperty,False,Individual Human,NaN,"[LLFS data dictionary, LLFS data dictionary, L...",NaN
6,172,taxon,Taxonomic identifier. The National Center for ...,"Unknown,Not collected,Not applicable",NaN,dataProperty,True,Individual nonHuman,NaN,"[https://www.ncbi.nlm.nih.gov/taxonomy, http:/...",NaN
7,173,speciesName,The scientific name of the species (typically ...,"Unknown,Not collected,Not applicable",NaN,dataProperty,True,Individual nonHuman,NaN,[https://www.ebi.ac.uk/ols4/ontologies/edam/te...,NaN
8,174,commonName,The biological species common name the individ...,"Not collected,Not applicable",NaN,dataProperty,True,Individual nonHuman,NaN,[https://www.ebi.ac.uk/ols4/ontologies/edam/te...,NaN
9,219,TRUE,Indicate the disease or condition.,"Other,Unknown,Not collected,Not applicable",diagnosis,validValue,False,Individual Human,NaN,"[LLFS data dictionary, LLFS data dictionary, L...",NaN


In [108]:
with pd.option_context('display.max_colwidth', None):
    display(dm_test['Source'])

0                                                                                                                                                                   [http://purl.obolibrary.org/obo/NCIT_C13018]
1                                                                                                                                                                   [http://purl.obolibrary.org/obo/NCIT_C12801]
2                                                   [https://www.ebi.ac.uk/ols4/ontologies/ccf/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FUBERON_0004537&lang=en&viewMode=PreferredRoots&siblings=false]
3    [https://bioportal.bioontology.org/ontologies/CCF/p=classes&conceptid=http%3A%2F%2Fpurl.org%2Fccf%2Fcell_type, https://www.ebi.ac.uk/ols4/ontologies/ccf/terms?iri=http%3A%2F%2Fpurl.org%2Fccf%2Fcell_type]
4                                                                                                                                            [https://wits.worldbank

In [109]:
dm.query('Attribute.str.contains("diagnosis")', engine='python')

,Unnamed: 0,Attribute,Description,Valid Values,DependsOn,Properties,Required,Parent,DependsOn Component,Source,Validation Rules
170,170,diagnosisStatus,Whether the individual has been diagnosed with...,"TRUE,FALSE,Unknown,Not collected,Not applicable",NaN,dataProperty,True,Individual Human,NaN,Sage Bionetworks,NaN
171,171,diagnosis,Indicate the disease or condition.,HPO and MONDO and MAXO codes or labels (not li...,NaN,dataProperty,False,Individual Human,NaN,"https://www.ebi.ac.uk/ols/ontologies/mondo,htt...",NaN


In [110]:
dm_test['Valid Values'] = dm_test['Valid Values'].str.split(",")

In [111]:
other_vvs = ['Other', 'Unknown', 'Not Available', 'Not Given']

## Main extraction


In [112]:
def sort_lists(l):
    try:
        return ','.join(sorted(l))
    except:
        return l

In [113]:
dm_test['extraction_status'] = ''

In [114]:
errors = []

for i, v in dm_test.iterrows():
    storage = dm_test.loc[i, "Valid Values"]
    for url in v['Source']:
        print(url)
        # for world countries
        if dm_test.at[i, "extraction_status"] == 'done':
            next
        else:
            if url == 'https://wits.worldbank.org/countryprofile/metadata/en/country/all':
                x = pd.read_excel(
                    io="http://wits.worldbank.org/data/public/WITSCountryProfile-Country_Indicator_ProductMetada-en.xlsx",
                    sheet_name="Country-Metadata"
                )
                
                dm_test.at[i, "Valid Values"] = sorted(storage +
                                                       x['Country Code'].tolist())

                dm_test.at[i, "extraction_status"] = 'done'

            elif "terms" in url and "ebi.ac.uk" in url:
                url = url_to_api_call(url)

                result = get_response(url)

                json_result = result.json()

                try:
                    url_descendents = json_result['_links']['hierarchicalDescendants']['href']

                    json_result = get_response(url_descendents).json()

                    terms_list = get_all_terms(url_descendents, json_result)

                    result = get_labels_from_terms_list(terms_list)

                    dm_test.at[i, "Valid Values"] = sorted(storage + result)

                    dm_test.at[i, "extraction_status"] = 'done'

                except Exception as e:
                    print(e)
                    next

            elif "http://purl" in url:
                try:
                    result = purl_main(url)

                    dm_test.at[i, "Valid Values"] = sorted(storage + result)

                    dm_test.at[i, "extraction_status"] = 'done'
                except Exception as e:
                    print(e)

            else:
                print("Error")
                errors.append(url)

        print("-" * 20)

errors

http://purl.obolibrary.org/obo/NCIT_C13018


<Response [200]>
<Response [200]>


100%|██████████| 18/18 [00:27<00:00,  1.55s/it]


Length of terms: 	359
--------------------
http://purl.obolibrary.org/obo/NCIT_C12801
<Response [200]>
<Response [200]>


100%|██████████| 26/26 [00:38<00:00,  1.50s/it]


...

100%|██████████| 2/2 [00:03<00:00,  1.79s/it]


Length of terms: 	22
--------------------
https://bioportal.bioontology.org/ontologies/CCF/p=classes&conceptid=http%3A%2F%2Fpurl.org%2Fccf%2Fcell_type
Error
--------------------
https://www.ebi.ac.uk/ols4/ontologies/ccf/terms?iri=http%3A%2F%2Fpurl.org%2Fccf%2Fcell_type
<Response [200]>
<Response [200]>


100%|██████████| 67/67 [01:51<00:00,  1.66s/it]


Length of terms: 	1321
--------------------
https://wits.worldbank.org/countryprofile/metadata/en/country/all
--------------------
LLFS data dictionary
Error
--------------------
LLFS data dictionary
Error
--------------------
LLFS data dictionary
Error
--------------------
https://www.ncbi.nlm.nih.gov/taxonomy
Error
--------------------
http://edamontology.org/data_1179
Error
--------------------
http://purl.obolibrary.org/obo/TAXRANK_0000010
<Response [200]>
'hierarchicalDescendants'
--------------------
https://www.ebi.ac.uk/ols4/ontologies/edam/terms?iri=http%3A%2F%2Fedamontology.org%2Fdata_1045
<Response [200]>
'hierarchicalDescendants'
--------------------
https://www.ebi.ac.uk/ols4/ontologies/edam/terms?iri=http%3A%2F%2Fedamontology.org%2Fdata_1874
<Response [200]>
'hierarchicalDescendants'
--------------------
LLFS data dictionary
Error
--------------------
LLFS data dictionary
Error
--------------------
LLFS data dictionary
Error
--------------------


['https://bioportal.bioontology.org/ontologies/CCF/p=classes&conceptid=http%3A%2F%2Fpurl.org%2Fccf%2Fcell_type',
 'LLFS data dictionary',
 'LLFS data dictionary',
 'LLFS data dictionary',
 'https://www.ncbi.nlm.nih.gov/taxonomy',
 'http://edamontology.org/data_1179',
 'LLFS data dictionary',
 'LLFS data dictionary',
 'LLFS data dictionary']

In [117]:
dm_test.loc[5:9, 'Valid Values'] = ''

In [118]:
dm_test.loc[4, 'Validation Rules'] = ''

In [120]:
dm_test['Valid Values'] = dm_test['Valid Values'].apply(
    lambda x: sort_lists(x))
    
dm_test['Source'] = dm_test['Source'].apply(
    lambda x: sort_lists(x))

In [121]:
dm_test

,Unnamed: 0,Attribute,Description,Valid Values,DependsOn,Properties,Required,Parent,DependsOn Component,Source,Validation Rules,extraction_status
0,143,organ,Indicate the organ the specimen is from,"Abdominal Skin,Accessory Lacrimal Gland,Adrena...",NaN,dataProperty,True,Biospecimen human,NaN,http://purl.obolibrary.org/obo/NCIT_C13018,NaN,done
1,144,tissue,Indicate the tissue the specimen is from,"Abdominal Peritoneum,Accessory Tissue,Acinar S...",NaN,dataProperty,True,Biospecimen human,NaN,http://purl.obolibrary.org/obo/NCIT_C12801,NaN,done
2,147,specimenSource,Indicate the biological area of the specimen,"Not applicable,Not collected,Other,Unknown,alv...",NaN,dataProperty,True,Biospecimen human,NaN,https://www.ebi.ac.uk/ols4/ontologies/ccf/term...,NaN,done
3,149,cellType,Indicate the cell type.,"A2 amacrine cell,Astro L1 FGFR3 SERPINI2 prima...",NaN,dataProperty,True,Biospecimen human,NaN,https://bioportal.bioontology.org/ontologies/C...,NaN,done
4,162,countryCode,Indicate the geographic region for the individ...,"000,004,008,010,012,016,020,024,028,031,032,03...",NaN,dataProperty,True,Individual Human,NaN,https://wits.worldbank.org/countryprofile/meta...,,done
5,171,diagnosis,Indicate the disease or condition.,,NaN,dataProperty,False,Individual Human,NaN,"LLFS data dictionary,LLFS data dictionary,LLFS...",NaN,
6,172,taxon,Taxonomic identifier. The National Center for ...,,NaN,dataProperty,True,Individual nonHuman,NaN,"http://edamontology.org/data_1179,http://purl....",NaN,
7,173,speciesName,The scientific name of the species (typically ...,,NaN,dataProperty,True,Individual nonHuman,NaN,https://www.ebi.ac.uk/ols4/ontologies/edam/ter...,NaN,
8,174,commonName,The biological species common name the individ...,,NaN,dataProperty,True,Individual nonHuman,NaN,https://www.ebi.ac.uk/ols4/ontologies/edam/ter...,NaN,
9,219,TRUE,Indicate the disease or condition.,,diagnosis,validValue,False,Individual Human,NaN,"LLFS data dictionary,LLFS data dictionary,LLFS...",NaN,


# Update data model

In [78]:
new_csv_name = "C:/Users/nlee/Documents/Projects/ELITE-DCC/ELITE-data-models/models/EL_data_model_v3.1.0.csv"

In [79]:
for i, v in dm_test.iterrows(): 
    old_index = v.pop('Unnamed: 0')
    dm.loc[old_index, :] = v

In [80]:
dm = dm.drop(columns=['Unnamed: 0'])

In [81]:
# write out new data model
dm.to_csv(new_csv_name)

# Extra

In [ ]:
def descendants_workflow(term):
    """ """

    # get descendants
    url = term['_links']['hierarchicalDescendants']['href']
    d = get_response(url)
    json_response = d.json()

    # gather all terms
    terms_list = get_all_terms(url, json_response)

    # extract all terms
    valid_values = get_labels_from_terms_list(terms_list)

    return valid_values

In [ ]:
def get_value_key(term):

    # get key for valid values in annotation dictionary
    r = re.compile(".*value", flags=re.IGNORECASE)

    new_list = list(filter(r.match, list(
        term['annotation'].keys())))

    if len(new_list) == 1:
        new_key = new_list[0]

        return new_key
    else:
        print("Too many keys in annotation dictionary")
        print(len(new_list))

In [ ]:
def get_valid_values(term, new_key):
    """ Get the purls for the valid values """
    return term['annotation'][new_key]

In [ ]:
def get_vv_label(iri):

    response = get_response(base_url, params={'iri': iri})

    # assuming there is only one term in the response
    term = simplify_response(response)

    label = term['label']

    return label

In [ ]:
def value_workflow(term):
    """ """

    # find the key that corresponds to all the values possible
    new_key = get_value_key(term)

    # get the iri purls from the response
    vvs = get_valid_values(term, new_key)

    # create list of valid values
    vvs_list = [get_vv_label(iri) for iri in vvs]

    return vvs_list

In [ ]:
def extract_values_wf(term):
    if 'hierarchicalDescendents' in term['_links'].keys():
        # work through descendants workflow
        valid_values = descendants_workflow(term)
    else:
        valid_values = value_workflow(term)  # work through value workflow

    return valid_values

In [ ]:
def main(purl):

    response = get_response(base_url, params={'iri': purl})

    term = simplify_response(response)

    valid_values = value_workflow(term)

    # check_descendents(...)

    return final

In [ ]:
base_url = 'http://www.ebi.ac.uk/ols4/api/terms'

addition = "/hierarchicalDescendants"

# to use with purl addresses
# v1 = requests.get(test, params={'iri': vvs[0]})

# v1.json()

# v1.json()['_embedded']['terms'][0]['label']

In [ ]:
# url = "https://www.ebi.ac.uk/ols4/api/ontologies/ms/terms/http%253A%252F%252Fpurl.obolibrary.org%252Fobo%252FMS_1000031"

# url = ''.join([url, addition])

In [ ]:
# response = get_response(url)

# d = response.json()

In [ ]:
# get all pages
pages = d['page']['totalPages']
size = d['page']['size']

terms = []

for i in tqdm(range(pages)):
    response = requests.get(url, params={'page': i, 'size': size})
    d = response.json()
    for term in d['_embedded']['terms']:
        terms.append(term)

In [ ]:
page = 1
requests.get(
    f"https://www.ebi.ac.uk/ols4/api/ontologies/ms/terms/http%253A%252F%252Fpurl.obolibrary.org%252Fobo%252FMS_1000031/hierarchicalDescendants?page={page}&size={size}").json()

In [ ]:
vv = []

for t in terms:
    if t['has_children'] == False:
        vv.append(t)

labels = []
for i in vv:
    labels.append(i['label'])

labels = sorted(np.unique(labels))